# Unit 2

## Automatic Few-Shot Learning with DSPy

## Implementing Your First Few-Shot Optimizer

## Self Generated Examples with BootstrapFewShot

## Finding Optimal Examples with Random Search

## Dynamic Example Selection with KNNFewShot

## Dynamic Example Selection with KNNFewShot